In [1]:
import gradio as gr
import joblib
import pytesseract
import pandas as pd
from PIL import Image
import numpy as np

c:\Users\vuvee\anaconda3\envs\dev\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the trained KNN model
knn_model = joblib.load('ocr_knn_model.pkl')

In [3]:
# Define the function to process the image
def process_image(image):
    """
    Process the uploaded image, extract text using OCR, and classify document numbers.
    """
    # Convert the uploaded image to a format pytesseract can process
    image = Image.fromarray(np.array(image))

    # Perform OCR
    ocr_data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)
    final_df = pd.DataFrame(ocr_data)

    # Compute Bounding Box Details
    final_df['bottom'] = final_df['top'].astype(int) + final_df['height'].astype(int)
    final_df['right'] = final_df['left'].astype(int) + final_df['width'].astype(int)

    # Select feature columns
    feature_columns = ["left", "top", "width", "height", "bottom", "right"]
    X_test = final_df[feature_columns].astype(int)

    # Predict the labels for the text
    final_df["predicted_label"] = knn_model.predict(X_test)

    # Gather all records labeled as 1 and convert the list into a string
    address = ' '.join(final_df.loc[final_df['predicted_label'] == 1, 'text'].to_list())

    return address

In [6]:
# Create Gradio Interface
interface = gr.Interface(
    fn=process_image,
    inputs=gr.Image(type="numpy"),
    outputs="text",
    title="Document OCR & Extraction",
    description="Upload an image to extract address using OCR and KNN classification."
)

In [7]:
# Run the Gradio app
if __name__ == "__main__":
    interface.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
